# Pivot

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'a':['John', 'Maria', 'Boby'],
    'b':['Maestria', 'Pregrado', 'Pregrado'],
    'c':[31, 23, 22]
}
)
df

In [ ]:
df = df.pivot(index='a', columns='b', values='c')
df

# GENERAR UN CSV

In [ ]:
import pandas as pd

dic = {'pares': [0, 2, 4, 6], 'impares': [1, 3, 5, 7]}

df = pd.DataFrame(dic)

## Guarda datos en CSV:
df.to_csv('num.csv', header=False, index=False)

In [ ]:
df

In [ ]:
from datetime import datetime as dt

## Guarda datos en CSV con un nombre variable:
fecha = str(dt.now()).split('.')[0]
file = f'prueba_{fecha}.csv'
df.to_csv(file, index=False)

In [ ]:
str(dt.now()).split('.')[0]

_______________________________________________________________

# Folium para visualizar datos sobre mapas

In [ ]:
# import sys
# !{sys.executable} -m pip install folium

In [ ]:
## Primer mapa con folium

import folium

mapa = folium.Map(location=[40.4320, 116.5703], zoom_start=20)
mapa

In [ ]:
## Segundo mapa con folium
## centra el mapa en la coordenada,entre mas grande el zoom 
## mas acercamiento

mapa = folium.Map(location=[40.4320, 116.5703], zoom_start=10, tiles='Cartodb dark_matter')
mapa

In [ ]:
## Tercer mapa con folium
## centra el mapa en la coordenada,entre mas grande el 
## zoom mas acercamiento


import folium as f
import json

lat = 35.762887
long = 84.083132

pais_geojson = json.load(open("World_Countries_(Generalized).geojson"))
print(type(pais_geojson))
mapa1 = f.Map(location=[lat, long], zoom_start=2)

f.GeoJson(pais_geojson, name="geojson").add_to(mapa1)
f.LayerControl().add_to(mapa1)
mapa1

In [ ]:
### GRAFICAR LOS PAISES SEPARADOS
import folium as f
import json

MO_LAT = 35.762887
MO_LONG = 84.083132

county_geojson = json.load(open("World_Countries_(Generalized).geojson"))

m= f.Map( location=[MO_LAT, MO_LONG], zoom_start=2 )

f.GeoJson( county_geojson, name='geojson').add_to(m)
f.LayerControl().add_to(m)
m

_______________________________________________________

## Data-Análisis con Pandas: Ejemplo Covid

###  Data sobre vacunas covid:
####  https://github.com/owid/covid-19-data/tree/master/public/data/vaccinations

In [ ]:
import folium #libreria de los mapas
import pandas as pd

def write_csv(df, file):
    df.to_csv(file)
    
def open_csv(file):
    return pd.read_csv(file)

In [ ]:
###############  SACAR Y ORGANIZAR LA DATA PARA TRABAJAR  ###############
# Info de vacunas diarias por país

file = "vaccinations.csv"
df = open_csv(file)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# # Método 1 para crear una nueva tabla

# ### En lugar de eliminar 12 columnas, extraemos las 4 que requierimos
# s_date   = df["date"]
# s_entity = df["location"]
# s_code   = df["iso_code"]
# s_vacs   = df["daily_vaccinations"]

# print( type(s_date.iloc[0]), s_date.iloc[0] )
# s_date = pd.to_datetime(s_date) # convierto en formato datetime
# s_date

In [ ]:
# ###############  CREAR EL DATAFRAME  ###############
# ### creo el diccionario de series
# data = {"date":s_date, "entity":s_entity, "code":s_code, "dayly_vacs":s_vacs}
# df = pd.DataFrame(data) # convierto el diccionario en dataframe
# df.set_index("date", inplace=True) # columna date será el index
# df.head()

In [ ]:
# Método 2 para crear una nueva tabla
df = df.iloc[:, [0,1,2,8]]

In [ ]:
df

In [ ]:
df.set_index('date', inplace=True)
df

In [ ]:
### descartamos los valores repetidos y los contamos una sola vez
print( "Total de paises:", len(df["location"].unique()) ) 
### 235 es el número de locaciones que aparecen en la data

In [ ]:
###############  FILTRO POR PAISES (GroupBy)  ###############
covid = df.groupby(['location'])#creo grupos para cada locación
#agrupa por key:value, donde key son los grupos de entity 
# (el nombre de cada locacion)
covid

In [ ]:
for key, group in covid:
    print('clave: ', key, '\size: ', len(group))
    print(group.head())
    print("\n")
    if "Andorra" in key: break

In [ ]:
#####  PROCESAMEINTO DE DATOS Y REQUERIMIENTOS  ######
total = covid.sum() #sumo el total de daily_vaccinations de cada 
## locación, resulta en el numero total de personas vacunadas

total

In [ ]:
## la columna 'iso_code' tambien se suma, resulta en:
## AFGAFGAFGAFGAFGAFGAFGAFGAFGAFGAFGAFGAFGAFGAFGA...
total.drop('iso_code', axis=1, inplace=True)
total

In [ ]:
##### CAMBIAR EL NOMBRE DE LA COLUMNA VACCCINATIONS

total.rename(columns={'daily_vaccinations':'total_vacs'}, inplace=True)
total

In [ ]:
## Almacenamos la información del DataFrame en un .csv

write_csv(total, 'total_vacs.csv')

In [ ]:
### Necesitamos la geo_data: mapa dividido por paises, 
### info del .geojson encontrado en:
### https://hub.arcgis.com/datasets/esri::world-countries-generalized/explore?location=-1.589978%2C68.791204%2C1.00


import folium as f
import json

file_json = "World_Countries_(Generalized).geojson"
data_geojson = json.load(open(file_json))

# Generando Mapa:
lat = 35.762887
long = 84.083132
mapa1 = f.Map(location=[lat, long], zoom_start=2)

# Agregando información al mapa:
f.Choropleth(geo_data=data_geojson,
                  data=total,
                  columns=(total.index, "total_vacs"),
                  key_on='properties.COUNTRY',
                  fill_color="Greens",
                  fill_opacity=0.7,
                  line_opacity=0.5
).add_to(mapa1)

folium.LayerControl().add_to(mapa1)
mapa1

In [ ]:
total.loc['Russia']

In [ ]:
### GRAFICAR EL MAPA DE CALOR
import folium
import json

def open_json(file):
    json_data = json.load(open(file))
    return json_data  


##### GRAFICAR COMO MAPA

## https://python-visualization.github.io/folium/

center=[35.762887,84.083132] #CENTRO DEL MAPA
m=folium.Map(location=center,zoom_start=2,
             max_bounds=True,
             min_zoom=1,
             min_lat=-84,
             max_lat=84,
             min_lon=-175,
             max_lon=187,
            )

#Necesitamos la geo_data :es el mapa dividido por paises, sacamos esa  info del .geojson encontrado en el siguiente link
#https://hub.arcgis.com/datasets/esri::world-countries-generalized/explore?location=-1.589978%2C68.791204%2C1.00

file_json="World_Countries_(Generalized).geojson"
data_geojson =open_json(file_json)

folium.Choropleth(geo_data=data_geojson,
                  data=total,
                  columns=(total.index,"total_vacs"),
                  key_on="properties.COUNTRY",
                  fill_color="RdYlGn",
                  fill_opacity=0.7,
                  line_opacity=0.5
).add_to(m)

folium.LayerControl().add_to(m)
m